In [1]:
# import pandas as pd

# df = pd.read_parquet('data/daily.parquet', engine='pyarrow')
# df.to_json('data/daily.json',orient='records', lines=True)

In [20]:
import json
import pickle
import torch
import faiss
import numpy as np
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv('.env')

True

In [21]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
api_key = os.getenv("GEMINI_API_KEY")
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

In [22]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=os.getenv("GEMINI_API_KEY"))
workflow = StateGraph(state_schema=MessagesState)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly knowledgeable and responsible AI doctor designed to analyze symptoms and provide general health guidance. Your primary role is to help users assess whether their symptoms require a consultation with a medical professional."
            "You should:"
            "Ask clarifying questions to understand the symptoms better."
            "Provide general health advice based on common medical knowledge."
            "Identify lifestyle habits or behaviors that may contribute to discomfort and suggest changes."
            "Never suggest specific medications, treatments, or dosages."
            "If symptoms are severe, unusual, or persistent, strongly recommend consulting a medical professional."
            "Avoid diagnosing medical conditions definitively—frame responses as guidance rather than medical conclusions."
            "Use a calm, reassuring, and professional tone to make users feel at ease."
            "Your goal is to empower users with knowledge about their symptoms while ensuring they seek professional help when necessary."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def input_func(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", input_func)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [23]:
# medical_meadow_health_advise
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return [entry['input'] + " " + entry['output'] for entry in data]

In [24]:
# Chunk the Data
def chunk_data(data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    chunks = splitter.split_text("\n".join(data))
    with open('data/chunks/high_fi.pkl', 'wb') as f:
        pickle.dump(chunks, f)
    return chunks

# Generate Embeddings
def embed(chunks):
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=chunks
    )
    tensor = torch.tensor(result['embedding'], dtype=torch.float32)
    return tensor

# Store Embeddings in FAISS
def store_embeddings(tensor):
    embeddings = np.array([embedding.cpu().numpy() for embedding in tensor])
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, "data/vectors/high_fi.faiss")
    return index

# Search for Relevant Chunks
def search_embeddings(query_embedding, chunks, k=3):
    query_embedding_array = np.expand_dims(np.array(query_embedding, dtype='float32'), axis=0)
    index = faiss.read_index("data/vectors/high_fi.faiss")
    distances, indices = index.search(query_embedding_array, k)
    matching_chunks = [chunks[i] for i in indices[0]]
    return matching_chunks

# Generate Response
def generate_response(matching_chunks, query, app):
    context = "\n".join(matching_chunks)
    # print("Context:\n", context)
    input_prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    input_message = [HumanMessage(content=input_prompt)]
    output = app.invoke({"messages": input_message}, config)
    return output["messages"][-1].content

# Analyze JSON Data
def analyze_json(json_file):
    data = load_json(json_file)
    print("file loaded")
    chunked = chunk_data(data)
    print("Chunked")
    # chunked = pickle.load(open('data\chunks\high_fi.pkl', 'rb'))
    # print("chunks loaded")
    embeddings = embed(chunked)
    print("Embedded")
    store_embeddings(embeddings)
    print("Embedding saved")
    # query_embedding = embed(query)
    # results = search_embeddings(query_embedding, chunked)
    # response = generate_response(results, query, app)
    # return response

In [25]:
json_file = "data\medical_meadow_health_advice.json"
ans = analyze_json(json_file)

file loaded
Chunked
Embedded
Embedding saved


In [1]:
query = "I am coughing a lot for more than one week. What should I do?"
query_embedding = embed(query)
chunked = pickle.load(open('data\chunks\high_fi.pkl', 'rb'))
results = search_embeddings(query_embedding, chunked)
response = generate_response(results, query, app)

print(response)

NameError: name 'embed' is not defined

In [10]:
query = "I have dry cough. It is severe. Laughing triggers it. "
query_embedding = embed(query)
chunked = pickle.load(open('data\chunks\high_fi.pkl', 'rb'))
results = search_embeddings(query_embedding, chunked)
response = generate_response(results, query, app)

Context:
 Whenever I have a really good laugh I end up coughing to the point that I have trouble catching my breath and taste blood. The cough usually last for several hours, and it happens every single time I get a good laugh. It has literally gotten to the point where my friends have started to warn me
I have a problem with severe dry cough attacks that seem to be triggered from sudden temperature changes and sometimes when eating. By temperature change I mean when I get in a cool vehicle when its hot outside is one example I have experienced. I have these attacks 3 or 4 times a day. There is
Hello Sir, I have dry cough problem since 1.5 months i consult many doctors but not diagnose very well and still suffer from this problem.I got cough after eating any meal and it continous whole day when i talk or take walk outside also so is there any serious problem Hello. Thank you for asking at


In [11]:
print(response)

A severe dry cough triggered by laughing is unusual and warrants immediate medical attention.  The fact that you're experiencing coughing to the point of breathlessness and tasting blood is a serious concern.  This is not something that can be safely addressed without a proper medical evaluation.

**Do not delay in seeking professional medical help.**  This symptom combination could indicate a variety of underlying issues that require prompt diagnosis and treatment.

While I cannot speculate on the cause,  possible explanations could range from an underlying respiratory condition to a more serious issue.  Only a doctor can determine the cause through a thorough examination, potentially including imaging studies or other tests.

**Please schedule an appointment with your doctor or go to an urgent care facility as soon as possible.**  This is not a condition to manage at home.  Your health and safety are paramount.
